source: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/gcn.py

In [2]:
# solved import problem with help of https://gist.github.com/ameya98/b193856171d11d37ada46458f60e73e7 


# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 7.9 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.4 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 709 kB 5.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/c

In [3]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv

In [4]:
parser = {}
parser['dataset'] = 'Cora'
parser['hidden_channels'] = 16
parser['lr'] = 0.01
parser['epochs'] =200
parser['use_gdc'] = 'store_true' # help='Use GDC'
parser['wandb'] = 'store_true' # help='Track experiment')
args = parser

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
init_wandb(name='GCN-Cora', lr=args['lr'], epochs= args['epochs'],
           hidden_channels=args['hidden_channels'], device=device)

In [5]:
path = osp.join('..', 'data', 'Planetoid')
dataset = Planetoid(path, args['dataset'], transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [6]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [7]:
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [8]:
sum(data.x)

tensor([0.7399, 1.6961, 3.4994,  ..., 0.3656, 3.2537, 0.6146])

In [21]:
max(data.y),min(data.y)

(tensor(6), tensor(0))

In [14]:
if args['use_gdc']:
    transform = T.GDC(
        self_loop_weight=1,
        normalization_in='sym',
        normalization_out='col',
        diffusion_kwargs=dict(method='ppr', alpha=0.05),
        sparsification_kwargs=dict(method='topk', k=128, dim=0),
        exact=True,
    )
    data = transform(data)

In [17]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, cached=True,
                             normalize=not args['use_gdc'])
        self.conv2 = GCNConv(hidden_channels, out_channels, cached=True,
                             normalize=not args['use_gdc'])

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [19]:
model = GCN(dataset.num_features, args['hidden_channels'], dataset.num_classes)
model, data = model.to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=args['lr'])  # Only perform weight-decay on first convolution.

In [20]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_weight)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)

In [21]:
@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_weight).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

In [23]:
best_val_acc = final_test_acc = 0
for epoch in range(1, args['epochs'] + 1):
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)

Epoch: 001, Loss: 57.3752, Train: 0.1429, Val: 0.1140, Test: 0.1030
Epoch: 002, Loss: 147.7970, Train: 0.1786, Val: 0.1900, Test: 0.1910
Epoch: 003, Loss: 51.2101, Train: 0.2857, Val: 0.3040, Test: 0.2810
Epoch: 004, Loss: 50.6918, Train: 0.4000, Val: 0.4620, Test: 0.4370
Epoch: 005, Loss: 47.9181, Train: 0.3429, Val: 0.4860, Test: 0.4650
Epoch: 006, Loss: 47.5617, Train: 0.3643, Val: 0.4820, Test: 0.4650
Epoch: 007, Loss: 41.7747, Train: 0.3714, Val: 0.4880, Test: 0.4620
Epoch: 008, Loss: 31.0296, Train: 0.3643, Val: 0.4800, Test: 0.4620
Epoch: 009, Loss: 23.9014, Train: 0.3857, Val: 0.4760, Test: 0.4620
Epoch: 010, Loss: 15.6696, Train: 0.4286, Val: 0.4880, Test: 0.4620
Epoch: 011, Loss: 10.8496, Train: 0.4786, Val: 0.5020, Test: 0.4890
Epoch: 012, Loss: 7.8014, Train: 0.5143, Val: 0.5460, Test: 0.5220
Epoch: 013, Loss: 4.8654, Train: 0.5286, Val: 0.5580, Test: 0.5350
Epoch: 014, Loss: 4.1973, Train: 0.5786, Val: 0.5520, Test: 0.5350
Epoch: 015, Loss: 3.6128, Train: 0.5643, Val: 0.53